In [3]:
from dotenv import load_dotenv
import streamlit as st
import json
import pandas as pd
import redis
import os
import matplotlib.pyplot as plt
import plotly.express as px

In [4]:
st.set_page_config(page_title="What's in my Kitchen?", page_icon="🥘")
st.title("🥘 What's in my Kitchen?")

2023-03-01 05:29:08.383 
  command:

    streamlit run C:\Users\shari\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\ipykernel_launcher.py [ARGUMENTS]


DeltaGenerator(_root_container=0, _provided_cursor=None, _parent=None, _block_type=None, _form_data=None)

In [5]:
load_dotenv()
r_host = os.environ.get('RD_HOST')
r_port = os.environ.get('RD_PORT')
r_pass = os.environ.get('RD_PASS')

In [6]:
print(os.getcwd())

d:\Codes\lynx_ai\pages


In [7]:
with open('D:\Codes\lynx_ai\pages\categories.txt') as c:
    data = c.read()
data = data.replace("'", "\"")
categories = json.loads(data)

In [8]:
categories

{'Bakery': ['durum', 'salt', 'sugar', 'bread'],
 'Canned goods': ['kidney beans', 'mushroom', 'tomato puree'],
 'Dairy': ['butter', 'cheese', 'egg', 'eggs', 'milk', 'yogurt'],
 'Fish': ['salmon', 'tuna'],
 'Fruits': ['apple', 'orange', 'tangerine'],
 'Grains': ['flour', 'musli', 'pasta', 'rice'],
 'Meat': ['beef', 'chicken', 'chicken breast', 'pork'],
 'Oil': ['cooking oil', 'olive oil'],
 'Spices': ['chilli powder',
  'garam masala',
  'garlic paste',
  'garlic powder',
  'ginger paste',
  'turmeric powder'],
 'Vegetables': ['carrot', 'garlic', 'onion', 'potatoes', 'tomato']}

In [9]:
@st.cache_data
def redis_call(host, port, password):

    r = redis.Redis(
        host=host,
        port=port,
        password=password)

    keys = r.keys()
    values = r.mget(keys)

    data = {}

    for key, value in zip(keys, values):
        data[f"{key.decode()}"] = f"{value.decode()}"

    return data

In [10]:
data = redis_call(r_host, r_port, r_pass)

In [11]:
data

{'Chicken Breast ': '{"quantity": "500", "unit": "gram"}',
 'Cooking Oil': '{"quantity": "500", "unit": "mililitre"}',
 'Sugar': '{"quantity": "1", "unit": "kilogram"}',
 'Ginger Paste': '{"quantity": "500", "unit": "gram"}',
 'Olive Oil': '{"quantity": "500", "unit": "mililitre"}',
 'Turmeric powder': '{"quantity": "100", "unit": "gram"}',
 'Chicken': '{"quantity": "1", "unit": "kilogram"}',
 'Kidney Beans': '{"quantity": "1", "unit": "kilogram"}',
 'Flour': '{"quantity": "500", "unit": "gram"}',
 'Rice': '{"quantity": "1", "unit": "kilogram"}',
 'Musli': '{"quantity": "1", "unit": "kilogram"}',
 'Tuna': '{"quantity": "250", "unit": "gram"}',
 'Tomato': '{"quantity": "1", "unit": "kilogram"}',
 'Carrot': '{"quantity": "1", "unit": "kilogram"}',
 'Garlic Paste': '{"quantity": "500", "unit": "gram"}',
 'Onion': '{"quantity": "1", "unit": "kilogram"}',
 'Garlic ': '{"quantity": "250", "unit": "gram"}',
 'Cheese': '{"quantity": "500", "unit": "gram"}',
 'Salt': '{"quantity": "250", "unit"

In [12]:
df = pd.DataFrame()

for key, value in data.items():
    if key != 'key':
        temp_df = pd.DataFrame.from_dict(json.loads(value), orient='index').T
        temp_df.index = [key]
        df = pd.concat([df, temp_df])
    else:
        pass

st.dataframe(df)

DeltaGenerator(_root_container=0, _provided_cursor=None, _parent=None, _block_type=None, _form_data=None)

In [13]:
def categorize(categories, grocery_data):
    # create new column
    grocery_data['category'] = ''

    # categorize
    for index, _ in grocery_data.iterrows():
        item_name = index.lower()
        for category, keywords in categories.items():
            for keyword in keywords:
                if keyword in item_name:
                    grocery_data.at[index, 'category'] = category
                    break

    return grocery_data.sort_values(by='category')

In [14]:
st.dataframe(categorize(categories, df))

DeltaGenerator(_root_container=0, _provided_cursor=None, _parent=None, _block_type=None, _form_data=None)

In [15]:
categorize(categories, df).head()

,quantity,unit,category
Sugar,1,kilogram,Bakery
Durum,2,Packet,Bakery
Salt,250,gram,Bakery
Mushroom,4,Cans,Canned goods
Kidney Beans,1,kilogram,Canned goods


In [16]:
def unit_conversion(dataFrame):

    # Convert Kilogram to gram
    dataFrame.loc[dataFrame['unit'] == 'Kilogram', ['quantity', 'unit']] = dataFrame.loc[dataFrame['unit'] == 'Kilogram', ['quantity', 'unit']].replace(
        {'quantity': {value: value*1000 for value in dataFrame.loc[dataFrame['unit'] == 'Kilogram', 'quantity']}, 'unit': {'Kilogram': 'gram'}})

    # Convert Litre to millilitre
    dataFrame.loc[dataFrame['unit'] == 'Litre', ['quantity', 'unit']] = dataFrame.loc[dataFrame['unit'] == 'Litre', ['quantity', 'unit']].replace(
        {'quantity': {value: value*1000 for value in dataFrame.loc[dataFrame['unit'] == 'Litre', 'quantity']}, 'unit': {'Litre': 'MilliLitre'}})

    return dataFrame

In [17]:
unit_conversion(df).head()

,quantity,unit,category
Chicken Breast,500,gram,Meat
Cooking Oil,500,mililitre,Oil
Sugar,1,kilogram,Bakery
Ginger Paste,500,gram,Spices
Olive Oil,500,mililitre,Oil


In [18]:
st.dataframe(unit_conversion(df))

DeltaGenerator(_root_container=0, _provided_cursor=None, _parent=None, _block_type=None, _form_data=None)

In [33]:
def plotting(dataFrame, index=None):
    if index is None: 
        index = dataFrame.index
    fig = px.bar(dataFrame, x=index, y='quantity')
    fig.show()
    return fig

In [34]:
plotting(df, index=None)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

In [ ]:
st.plotting(df, index)

AttributeError: module 'streamlit' has no attribute 'plotting'

In [ ]:
import sys
print(sys.version)

3.10.10 (tags/v3.10.10:aad5f6a, Feb  7 2023, 17:20:36) [MSC v.1929 64 bit (AMD64)]


In [ ]:
def plotting(dataFrame, index):

    piece_df = dataFrame.loc[dataFrame['Unit'] == 'Piece']
    gram_df = dataFrame.loc[dataFrame['Unit'] == 'gram']
    milliLitre_df = dataFrame.loc[dataFrame['Unit'] == 'MilliLitre']

    if not piece_df.empty:
        p = piece_df.plot(kind='bar', x=index, y='Quantity')
        p.set_xlabel('Current Items')
        p.set_ylabel('Quantity (Pieces)')

    if not gram_df.empty:
        g = gram_df.plot(kind='bar', x=index, y='Quantity')
        g.set_xlabel('Current Items')
        g.set_ylabel('Quantity (gram)')

    if not milliLitre_df.empty:
        l = milliLitre_df.plot(kind='bar', x=index, y='Quantity')
        l.set_xlabel('Current Items')
        l.set_ylabel('Quantity (Millilitre)')

    plt.show()